In [ ]:
import os

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold

In [ ]:
import cv2
import torch

In [ ]:
dataset_path = Path('/content/gdrive/My Drive/ES100root') # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("labels_s/*.txt")) # all data in 'labels'

In [ ]:
yaml_file = '/content/gdrive/My Drive/ES100root/config.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)
    classes = classes['names']
cls_idx = sorted(classes)

In [ ]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [ ]:
for label in labels:
    lbl_counter = Counter()
    print(label)

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter[0]

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000000.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000001.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000002.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000003.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000004.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000005.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000006.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000007.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000008.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000009.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000010.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000011.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000012.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000013.txt
/content/gdrive/My Drive/ES100root/labels_s/P1_frame_000014.txt
/content/gdrive/My Drive/ES100root/label

In [ ]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

In [ ]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [ ]:
supported_extensions = ['.jpg', '.jpeg', '.png']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images' / 'train').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'images' / 'val').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'image_temp' / 'train_temp').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'image_temp' / 'val_temp').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'labels_temp' / 'P8_images').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'image_temp' / 'P1').rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val-134589')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [ ]:
sorted_image = sorted(images, key = lambda image: image.stem)

In [ ]:
label_counter = 0
for image in sorted_image:
  if image.stem in folds_df.index:
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(labels[label_counter], lbl_to_path / labels[label_counter].name)
    print(f"Image inputted from '{image.name}' and label inputted from '{labels[label_counter].name}'.")
    label_counter += 1

Image inputted from 'P1_frame_000000.png' and label inputted from 'P1_frame_000000.txt'.
Image inputted from 'P1_frame_000001.png' and label inputted from 'P1_frame_000001.txt'.
Image inputted from 'P1_frame_000002.png' and label inputted from 'P1_frame_000002.txt'.
Image inputted from 'P1_frame_000003.png' and label inputted from 'P1_frame_000003.txt'.
Image inputted from 'P1_frame_000004.png' and label inputted from 'P1_frame_000004.txt'.
Image inputted from 'P1_frame_000005.png' and label inputted from 'P1_frame_000005.txt'.
Image inputted from 'P1_frame_000006.png' and label inputted from 'P1_frame_000006.txt'.
Image inputted from 'P1_frame_000007.png' and label inputted from 'P1_frame_000007.txt'.
Image inputted from 'P1_frame_000008.png' and label inputted from 'P1_frame_000008.txt'.
Image inputted from 'P1_frame_000009.png' and label inputted from 'P1_frame_000009.txt'.
Image inputted from 'P1_frame_000010.png' and label inputted from 'P1_frame_000010.txt'.
Image inputted from '

In [ ]:
save_path_background = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val-backgrounds')
save_path_background.mkdir(parents=True, exist_ok=True)

for image in sorted_image:
  if image.stem not in folds_df.index:

    shutil.copy(image, save_path_background / image.name)
    print(f"Image '{image.name}' not in folds_df.")

Image 'P1_frame_000043.png' not in folds_df.
Image 'P1_frame_000044.png' not in folds_df.
Image 'P1_frame_000045.png' not in folds_df.
Image 'P1_frame_000128.png' not in folds_df.
Image 'P3_frame_000000.png' not in folds_df.
Image 'P3_frame_000001.png' not in folds_df.
Image 'P3_frame_000038.png' not in folds_df.
Image 'P3_frame_000039.png' not in folds_df.
Image 'P3_frame_000040.png' not in folds_df.
Image 'P3_frame_000047.png' not in folds_df.
Image 'P3_frame_000049.png' not in folds_df.
Image 'P3_frame_000052.png' not in folds_df.
Image 'P3_frame_000080.png' not in folds_df.
Image 'P3_frame_000081.png' not in folds_df.
Image 'P3_frame_000082.png' not in folds_df.
Image 'P3_frame_000083.png' not in folds_df.
Image 'P3_frame_000084.png' not in folds_df.
Image 'P3_frame_000085.png' not in folds_df.
Image 'P3_frame_000086.png' not in folds_df.
Image 'P3_frame_000087.png' not in folds_df.
Image 'P3_frame_000088.png' not in folds_df.
Image 'P3_frame_000089.png' not in folds_df.
Image 'P3_

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!scp -r /content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-backgrounds '/content/gdrive/My Drive/ES100root/'

cp: '/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-backgrounds' and '/content/gdrive/My Drive/ES100root/2024-03-19_5-Fold_Cross-val-backgrounds' are the same file


In [ ]:
model = YOLO('yolov8s-seg.pt', task = 'detect')

100%|██████████| 22.8M/22.8M [00:00<00:00, 316MB/s]


In [ ]:
ds_yamls_temp = ['/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-134589/split_1/split_1_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-134589/split_2/split_2_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-134589/split_3/split_3_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-134589/split_4/split_4_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-134589/split_5/split_5_dataset.yaml']

In [ ]:
results = {}

ksplit = 5

# Define your additional arguments here
batch = 16
project = 'kfold_demo'
epochs = 10

for k in range(ksplit):
    dataset_yaml = ds_yamls_temp[k]
    model.train(data=dataset_yaml, cfg = '/content/best_hyperparameters-2.yaml', epochs=epochs, batch=batch, project=project, optimizer = 'AdamW')  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=segment, mode=train, model=None, data=/content/gdrive/MyDrive/ES100root/2024-03-19_5-Fold_Cross-val-134589/split_1/split_1_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_demo, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

100%|██████████| 755k/755k [00:00<00:00, 39.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 191MB/s]


AMP: checks passed ✅


...

      10/10      7.28G      1.117      1.295      0.498      1.136          5        640: 100%|██████████| 72/72 [00:21<00:00,  3.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.29it/s]

                   all        285        282      0.964      0.972      0.984      0.733      0.964      0.972      0.984      0.728



10 epochs completed in 0.079 hours.
Optimizer stripped from kfold_demo/train5/weights/last.pt, 23.8MB
Optimizer stripped from kfold_demo/train5/weights/best.pt, 23.8MB

Validating kfold_demo/train5/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.18it/s]


                   all        285        282      0.964      0.972      0.984      0.733      0.964      0.972      0.984      0.728
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to kfold_demo/train5


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!scp -r /content/kfold_demo_134589 '/content/gdrive/My Drive/ES100root/runs'

In [ ]:
models_split1_134589 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134589/train/weights/best.pt')
models_split2_134589 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134589/train2/weights/best.pt')
models_split3_134589 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134589/train3/weights/best.pt')
models_split4_134589 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134589/train4/weights/best.pt')
models_split5_134589 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134589/train5/weights/best.pt')

In [ ]:
sources = ['/content/gdrive/My Drive/ES100root/Videos/CDW_4.6.23_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/CDW_11.16.22_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/CFY_6.6.22_RUL_PA1_1fps.mp4',
           '/content/gdrive/My Drive/ES100root/Videos/CFY_6.13.22_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/LS_5.15.23_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4',
           '/content/gdrive/My Drive/ES100root/Videos/US_2.15.23_RUL_PA1_1fps.mp4']

patient_num = ['1','3','4','5','7','8','9']
patient_counter = 0
for source in sources:
  results_split1_134589 = models_split1_134589(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_134589", name="split1")  # generator of Results objects
  results_split2_134589 = models_split2_134589(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_134589", name="split2")
  results_split3_134589 = models_split3_134589(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_134589", name="split3")
  results_split4_134589 = models_split4_134589(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_134589", name="split4")
  results_split5_134589 = models_split5_134589(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_134589", name="split5")

  counter = 0
  for r in results_split1_134589:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_134589/split1_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split2_134589:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_134589/split2_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split3_134589:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_134589/split3_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split4_134589:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_134589/split4_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split5_134589:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_134589/split5_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1


  patient_counter += 1

Streaming output truncated to the last 5000 lines.
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0., 0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0., 0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0., 0., 0.], device='cuda:0')
clss
tensor([0., 0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device